# <a id='toc1_'></a>[Restaurants Analysis](#toc0_)

# <a id='toc2_'></a>[Context](#toc0_)

You have decided to open a little coffee shop administrated with robots in Los Ángeles. The project is promising but expensive, so you and your partners decided to attract investors. They are interested in the actual condition of the market, are you going to maintain your success when the novelty of the robot waiters disappears?

The task is to prepare a market study. You have data from open sources about restaurants in LA. 

# <a id='toc0_'></a>[Table of Contents](#toc0_)
  
- [Restaurants Analysis](#toc1_)    
- [Context](#toc2_)    
- [Table of Contents](#toc0_)    
- [Introduction](#toc4_)    
- [Step 1: Download and preparation of the data](#toc5_)    
- [Step 2. Data analysis](#toc6_)    
  - [Proportion of the distinct type of establishments](#toc6_1_)    
  - [Proportion of the different type of establishment that are part of a chain or not](#toc6_2_)    
  - [Witch type of establishment is usually a chain?](#toc6_3_)    
  - [Which is more common for chain establishments: a lot establishments with a low number of seats or few establishments with a lot of seats?](#toc6_4_)    
  - [The mean number of seats for each type of establishments](#toc6_5_)    
  - [Top 10 streets for number of restaurants](#toc6_6_)    
  - [Streets that only have one restaurant](#toc6_7_)    
  - [For each restaurant in the top 10, analyze the distribution of number of seats, is there a tendency?](#toc6_8_)    
- [Conclusions](#toc7_)    
    - [Presentation](#toc7_1_1_)    

# <a id='toc4_'></a>[Introduction](#toc0_)

For this project we have to analyze a dataset with the purpose of investigate the market behavior of the restaurants in LA. 

This project is segmented in the following parts:

1. Download and preparation of the data

2. Data analysis

3. Presentation of the results

As the project is develop, we are going to explain each step. 

Without further clarification, let's begin with the procedure.

# <a id='toc5_'></a>[Step 1: Download and preparation of the data](#toc0_)

In this step we are going to load the libraries that we are going to use in develop of this project. Also we are going to load the dataset and review if the data have the correct data type for each column and verify if the data have missing or duplicated values. So lets continue with the procedure.

In [1]:
# Loading the libraries
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go 
import matplotlib.pyplot as plt
import re

Now we can proceed with loading and processing the data.

In [2]:
# Loading the data
data = pd.read_csv('./datasets/rest_data_us_upd.csv')

# Data inspection
print(data.describe())
print(data.info())
data.head(5)

                 id       number
count   9651.000000  9651.000000
mean   16611.000000    43.695161
std     2786.148058    47.622874
min    11786.000000     1.000000
25%    14198.500000    14.000000
50%    16611.000000    27.000000
75%    19023.500000    46.000000
max    21436.000000   229.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9651 entries, 0 to 9650
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           9651 non-null   int64 
 1   object_name  9651 non-null   object
 2   address      9651 non-null   object
 3   chain        9648 non-null   object
 4   object_type  9651 non-null   object
 5   number       9651 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 452.5+ KB
None


,id,object_name,address,chain,object_type,number
0,11786,HABITAT COFFEE SHOP,3708 N EAGLE ROCK BLVD,False,Cafe,26
1,11787,REILLY'S,100 WORLD WAY 120,False,Restaurant,9
2,11788,STREET CHURROS,6801 HOLLYWOOD BLVD 253,False,Fast Food,20
3,11789,TRINITI ECHO PARK,1814 W SUNSET BLVD,False,Restaurant,22
4,11790,POLLEN,2100 ECHO PARK AVE,False,Restaurant,20


As we can see in the results of the first exploration of the data, the column `id` have a wrong type of data, the id it's supposed to be a string object, the column `chain` it's need to change the data type as well, we are going to change it to boolean, also we can see that the `chain` column has missing data. Before changing the data type of the columns, we are going to resolve the problem of the missing data.  

In [3]:
# Reviewing missing content of the column "chain"
data[data['chain'].isna()]


,id,object_name,address,chain,object_type,number
7408,19194,TAQUERIA LOS 3 CARNALES,5000 E WHITTIER BLVD,NaN,Restaurant,14
7523,19309,JAMMIN JIMMY'S PIZZA,1641 FIRESTONE BLVD,NaN,Pizza,1
8648,20434,THE LEXINGTON THEATER,129 E 3RD ST,NaN,Restaurant,35


In [4]:
# Reviewing the values of the column "chain"
data['chain'].unique()

array([False, True, nan], dtype=object)

As we can see, they are only three rows of the 9651 of all the dataset, so we can drop this rows without affecting the results of the analysis. 

In [5]:
# dropping the rows with missing information of the column "chain"
data = data.dropna(subset=['chain'])

# Verifying
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9648 entries, 0 to 9650
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           9648 non-null   int64 
 1   object_name  9648 non-null   object
 2   address      9648 non-null   object
 3   chain        9648 non-null   object
 4   object_type  9648 non-null   object
 5   number       9648 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 527.6+ KB


Now we proceed to change the data types of the columns `id` and `chain`.

In [6]:
# Changing the data types of the columns "id" and "chain"
data['id'] = data['id'].astype('object')
data['chain'] = data['chain'].astype(bool)

# Verifying
data.dtypes

id             object
object_name    object
address        object
chain            bool
object_type    object
number          int64
dtype: object

Reviewing the name of the columns we see that the columns `object_name`, `object_type` and `number` are't names that describes the content of their column, so we are going to change it for `restaurant_name`, `restaurant_type` and `seats`.

In [7]:
# Changing columns name
data = data.rename(columns={'object_name': 'restaurant_name'})
data = data.rename(columns={'object_type': 'restaurant_type'})
data = data.rename(columns={'number': 'seats'})

# Verifying
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9648 entries, 0 to 9650
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               9648 non-null   object
 1   restaurant_name  9648 non-null   object
 2   address          9648 non-null   object
 3   chain            9648 non-null   bool  
 4   restaurant_type  9648 non-null   object
 5   seats            9648 non-null   int64 
dtypes: bool(1), int64(1), object(4)
memory usage: 461.7+ KB


As we can see, the only columns that could have duplicates are `id`, `restaurant_name` and `address`. let's confirm this.

In [8]:
# Searching for duplicate items
data[['id', 'restaurant_name', 'address']].duplicated().sum()

0

They are no duplicates in the dataset. Finally we are going to establish the columns `restaurant_name` and `address` in upper characters.

In [9]:
# Convert every character in upper for the columns "restaurant_name" and "address" 
data['restaurant_name'] = data['restaurant_name'].str.upper()
data['address'] = data['address'].str.upper()

Once we did this process we can continue with the analysis. 

# <a id='toc6_'></a>[Step 2. Data analysis](#toc0_)

In this step we are going to investigate the following cases:

* Proportion of the distinct type of establishments

* Proportion of the different type of establishments that are part of a chain and those that are not part of a chain.

* witch type of establishment is usually a chain?

* Which is more common for chain establishments: a lot establishment with a low number of seats or few establishments with a lot of seats? 

* The mean number of seats for each type of establishment

* The top 10 streets for number of restaurants

* The streets that only have one restaurant

* For each restaurant in the top 10, analyze the distribution of number of seats, is there a tendency?

let's begin. 

## <a id='toc6_1_'></a>[Proportion of the distinct type of establishments](#toc0_)

For this section we are going to group by type of establishment and plot the result.

In [10]:
# Grouping by establishment 
establishments = data.groupby('restaurant_type', as_index=False)['id'].count()
establishments.columns = ['type', 'number']

establishments = establishments.sort_values(by='number', ascending=False)

establishments

,type,number
5,Restaurant,7253
3,Fast Food,1066
2,Cafe,435
4,Pizza,319
1,Bar,292
0,Bakery,283


In [11]:
# Plot the result
fig = px.bar(establishments,
            x='type',
            y='number',
            title='Establishments type')
fig.show()

The "Restaurant" is the type of establishment that predominates over the others, followed by "Fast Food" and "Cafe".

## <a id='toc6_2_'></a>[Proportion of the different type of establishment that are part of a chain or not](#toc0_)

Now we are going to group by type of establishment and chain, then we are going to plot the results. 

In [12]:
# Grouping by establishment and chain
establishments_chain = data.groupby(['restaurant_type', 'chain'], as_index=False)['id'].count()
establishments_chain.columns = ['type', 'chain', 'number']

establishments_chain

,type,chain,number
0,Bakery,True,283
1,Bar,False,215
2,Bar,True,77
3,Cafe,False,169
4,Cafe,True,266
5,Fast Food,False,461
6,Fast Food,True,605
7,Pizza,False,166
8,Pizza,True,153
9,Restaurant,False,4961


In [13]:
# Plot the result
fig = px.bar(establishments_chain,
            x='type',
            y='number',
            color='chain',
            title="Establishments type and if it's part of a chain",
            barmode='group')
fig.show()

Usually the "restaurant" establishments are not in a chain, but the "Fast Food" and "Cafe" establishments are more likely came in chains.

## <a id='toc6_3_'></a>[Witch type of establishment is usually a chain?](#toc0_)

In this section we are going to calculate a percentage of the establishments that are usually came in chains. 

In [14]:
# Creation of a Dataframe with the number of establishments that are in a chain
chain_number = data[data['chain']==True].groupby('restaurant_type', as_index=False)['id'].count()
chain_number.columns = ['type', 'in_chain']
chain_number

,type,in_chain
0,Bakery,283
1,Bar,77
2,Cafe,266
3,Fast Food,605
4,Pizza,153
5,Restaurant,2292


In [15]:
# Merging the last Dataframe with the establishment Dataframe
establishments = pd.merge(establishments, chain_number, left_on='type', right_on='type')

establishments

,type,number,in_chain
0,Restaurant,7253,2292
1,Fast Food,1066,605
2,Cafe,435,266
3,Pizza,319,153
4,Bar,292,77
5,Bakery,283,283


Now we calculate the percentage of the establishments that came in a chain.

In [16]:
# Calculating the percentage
establishments['chain_percentage'] = establishments['in_chain']/establishments['number']

establishments = establishments.sort_values(by='chain_percentage', ascending=False)

establishments

,type,number,in_chain,chain_percentage
5,Bakery,283,283,1.000000
2,Cafe,435,266,0.611494
1,Fast Food,1066,605,0.567542
3,Pizza,319,153,0.479624
0,Restaurant,7253,2292,0.316007
4,Bar,292,77,0.263699


In [17]:
# Plot the result

fig = px.bar(establishments,
            x='type',
            y='chain_percentage',
            title='Percentage of establishments that came in a chain')
fig.show()

Usually all the `Bakery` establishments come with a chain, followed by `Cafe` and `Fast Food` establishments.

## <a id='toc6_4_'></a>[Which is more common for chain establishments: a lot establishments with a low number of seats or few establishments with a lot of seats?](#toc0_)

To answer this section we need to group by establishment that are in a chain and calculate the mean number of seats of each type. 

In [18]:
# Grouping by establishment 
establish_seats = data[data['chain'] == True][['restaurant_type', 'seats']]

establish_seats

,restaurant_type,seats
8,Fast Food,1
10,Restaurant,38
11,Bakery,17
12,Restaurant,133
14,Bakery,26
...,...,...
9629,Bakery,10
9636,Restaurant,47
9639,Fast Food,135
9648,Fast Food,15


In [36]:
# Plot the result
fig = px.histogram(establish_seats,
                x='seats',
                title='Seats in Chain establishments')
fig.show()

As we can see, the chain establishments prefers a lot of establishments with a few number of seats.

## <a id='toc6_5_'></a>[The mean number of seats for each type of establishments](#toc0_)

Now we are going to calculate the mean number of seats for each establishment. 

In [20]:
# Grouping by establishment and calculating the mean number of seats
mean_seats = data.groupby('restaurant_type', as_index=False)['seats'].mean()
mean_seats.columns = ['type', 'mean_seats']

mean_seats = mean_seats.sort_values(by='mean_seats', ascending=False)

mean_seats

,type,mean_seats
5,Restaurant,48.048807
1,Bar,44.767123
3,Fast Food,31.837711
4,Pizza,28.545455
2,Cafe,25.000000
0,Bakery,21.773852


In [21]:
# Plot the results
fig = px.bar(mean_seats,
            x='type',
            y='mean_seats',
            title='Mean number of seats')
fig.show()

The three establishments that have the largest number of seats are:

* Restaurants with 48 mean seats
* Bar with 45 mean seats
* Fast Foods with 32 mean seats

## <a id='toc6_6_'></a>[Top 10 streets for number of restaurants](#toc0_)

For this sections we need to add a new column in our data to get only the street name. For this we need to section the `address` column and discard if the first section of numbers. 

In [22]:

data['street'] = data['address'].apply(lambda x: ' '.join(x.split()[1:]))

data.head(5)

,id,restaurant_name,address,chain,restaurant_type,seats,street
0,11786,HABITAT COFFEE SHOP,3708 N EAGLE ROCK BLVD,False,Cafe,26,N EAGLE ROCK BLVD
1,11787,REILLY'S,100 WORLD WAY 120,False,Restaurant,9,WORLD WAY 120
2,11788,STREET CHURROS,6801 HOLLYWOOD BLVD 253,False,Fast Food,20,HOLLYWOOD BLVD 253
3,11789,TRINITI ECHO PARK,1814 W SUNSET BLVD,False,Restaurant,22,W SUNSET BLVD
4,11790,POLLEN,2100 ECHO PARK AVE,False,Restaurant,20,ECHO PARK AVE


Now we are going to group by street and count the number of restaurants for each street. 

In [23]:
# Grouping by street and counting the restaurants
restaurantsByStreet = (data.
                    groupby(by='street', as_index=False).
                    agg({'id':'count', 'seats': 'mean'}))

# Renaming the columns 
restaurantsByStreet.columns = ['street', 'number', 'seats_mean']

# Sorting the values
restaurantsByStreet = restaurantsByStreet.sort_values(by='number', ascending=False)

#show the results
restaurantsByStreet.head(10)

,street,number,seats_mean
2675,W SUNSET BLVD,297,50.781145
2606,W PICO BLVD,289,40.764706
579,HOLLYWOOD BLVD,168,53.440476
2819,WILSHIRE BLVD,161,62.130435
1701,S VERMONT AVE,148,45.878378
1912,SANTA MONICA BLVD,146,32.390411
2162,W 3RD ST,145,43.931034
147,BEVERLY BLVD,135,44.770370
1292,S FIGUEROA ST,134,50.761194
752,MELROSE AVE,128,52.429688


In [24]:
# Plot the results
fig = px.bar(restaurantsByStreet.head(10),
            x='street',
            y='number',
            title='Top 10 restaurants streets')
fig.show()

As we can see, the streets that have a lot of restaurants are "W SUNSET BLVD" , "W PICO BLVD" and "HOLLYWOOD BLVD".

## <a id='toc6_7_'></a>[Streets that only have one restaurant](#toc0_)

Now we are going to count the streets that only have one restaurant.

In [25]:
# Counting the streets that only have one restaurant
oneRestaurantStreets = restaurantsByStreet[restaurantsByStreet['number'] == 1]['street'].count()

print("The number of streets that only have one restaurant is: ", oneRestaurantStreets)

The number of streets that only have one restaurant is:  2440


That's the 25% of all the stablishments that we have in this dataset, it seems that there are streets of preference to be more likely to open a restaurant.

## <a id='toc6_8_'></a>[For each restaurant in the top 10, analyze the distribution of number of seats, is there a tendency?](#toc0_)

We all ready have the mean seats for the top 10 streets that have a lot of restaurants, we only need to plot that result. 

In [26]:
# Plot the result
fig = px.bar(restaurantsByStreet.head(10),
            x='street',
            y='seats_mean',
            title="Mean seats for the top 10 streets of restaurants")
fig.show()

In [27]:
# Mean seats for the top 10 streets of restaurants
print("The mean number of seats for the top 10 streets of restaurants are: ",
    restaurantsByStreet['seats_mean'].head(10).mean())

The mean number of seats for the top 10 streets of restaurants are:  47.72778373568448


As we can see, the restaurants that are in these streets have the tendency of have a lot of seats. The mean number of seats for these restaurants is 48.

# <a id='toc7_'></a>[Conclusions](#toc0_)

Throughout the analysis we find the following behaviors:

* In Los Ángeles the establishments that are more likely to open are:
    * Restaurants (7253)
    * Fast Food (1066)
    * Cafe (435)
* The establishments that are more likely came in chain are:
    * Bakery (100%)
    * Cafe (61.14%)
    * Fast Food (56.75%)
* The chain business are more likely to open few establishments with a lot of seats. 
* The number of seats depends of the type of establishment, the mean number of seats for each type are: 
    * Restaurant -> 48 seats
    * Bar -> 44 seats
    * Fast Food -> 32 seats
    * Pizza -> 29 seats
    * Cafe -> 25 seats
    * Bakery -> 22 seats
* The streets that are more likely for open a establishment are: 
    * SUNSET BLVD
    * PICO BLVD
    * WILSHIRE BLVD
    * WESTERN AVE
    * FIGUEROA ST
    * OLYMPIC BLVD
    * VERMONT AVE
    * HOLLYWOOD BLVD
    * 3RD ST
    * SANTA MONICA BLVD
* The mean number of seats for these top 10 establishments are 45.

According to these findings we can conclude that the establishment type that is more common to open are the Restaurant with a lot of advantage above of the other types also, this kind of establishment have the more mean number of seats, but is more uncommon to find restaurants that came with a chain.

### <a id='toc7_1_1_'></a>[Presentation](./presentation/Restaurants%20Analysis.pdf) [&#8593;](#toc0_)